## 0. Import Modules and Data

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import pandas_datareader.data as web
import datetime as dt

In [51]:
import Portfolio as port # This is from the Portfolio.py file with the different classes defined

## 1. Set-up Portfolio Strategy

In [81]:
# Define start date and end date

start_date = '2015-01-01'

end_date = '2020-01-05'

In [4]:
# Define Shares

# Provide the name of the ticker and type (Equity or Bond)

voo = port.Share('VOO', 'Equity')
bnd = port.Share('BND', 'Bond')

In [5]:
voo.get_value(start_date)

247.09

In [6]:
voo.get_value('2019-10-31')

278.55

In [7]:
# Need a way to group the different shares together

shares_list = [voo, bnd]

shares_dict = {}

for share in shares_list:
    shares_dict[share] = share.type


In [8]:
shares_dict

{<Portfolio.Share at 0x7f68c8a81a58>: 'Equity',
 <Portfolio.Share at 0x7f68c8a817b8>: 'Bond'}

In [69]:
# Define Strategy

# Provide the equity distribution, the bond distriubtion, cash distribution, and the threshold
strat = port.Strategy(50,50,0,5)

In [95]:
# Run portfolio with set start and end dates

portfolio = port.Portfolio(shares_dict)

portfolio.initial_buy(500, strat, start_date)

In [96]:
portfolio.get_asset_values(start_date)

{'Equities': 250.0, 'Bonds': 250.0, 'Cash': 0}


In [97]:
portfolio.asset_split

{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}

In [98]:
# Run the portfolio over a series of months

time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

for day in time_period:
    print(day)
    portfolio.reinvest_divs(day)
    portfolio.get_asset_values(day)
    print(portfolio.asset_split)
    if portfolio.asset_split['Equities'] > strat.equity_distribution+strat.threshold:
        sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*(portfolio.asset_split['Equities']-strat.equity_distribution)
        sell_amt_per = sell_amt/len(portfolio.equities)
        for share in portfolio.equities: # sell equities and buy more bonds
            portfolio.sell(share, sell_amt_per, day)
        for share in portfolio.bonds:
            portfolio.buy(share, sell_amt_per, day)
               
    if portfolio.asset_split['Bonds'] > strat.bond_distribution+strat.threshold:
        sell_amt = (portfolio.asset_values['']+portfolio.asset_values['Bonds'])*(portfolio.asset_split['Bonds']-strat.bond_distribution)
        sell_amt_per = bond_amt/len(portfolio.bonds)
        for share in portfolio.bonds: # sell bonds and buy more equities
            portfolio.sell(sharsell_amt_per,share, day)
        for share in portfolio.bonds:
            portfolio.buy(sell_amt_per, share, day)
        
    

2015-01-01 00:00:00
{'Equities': 250.0, 'Bonds': 250.0, 'Cash': 0}
{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}
2015-01-02 00:00:00
{'Equities': 250.0, 'Bonds': 250.0, 'Cash': 0}
{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}
2015-01-03 00:00:00
{'Equities': 245.60774946921444, 'Bonds': 250.72595281306712, 'Cash': 0}
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-04 00:00:00
{'Equities': 245.60774946921444, 'Bonds': 250.72595281306712, 'Cash': 0}
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-05 00:00:00
{'Equities': 245.60774946921444, 'Bonds': 250.72595281306712, 'Cash': 0}
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-06 00:00:00
{'Equities': 243.19267515923568, 'Bonds': 251.45190562613425, 'Cash': 0}
{'Equities': 49.16513484755205, 'Bonds': 50.83486515244794, 'Cash': 0.0}
2015-01-07 00:00:00
{'Equities': 246.23142250530785, 'Bonds': 251.6031457955233, 'Cash': 0}
{'Equities': 49.

{'Equities': 257.370223089183, 'Bonds': 252.60958177784724, 'Cash': 0}
{'Equities': 50.46674802275524, 'Bonds': 49.53325197724476, 'Cash': 0.0}
2015-04-26 00:00:00
{'Equities': 257.370223089183, 'Bonds': 252.60958177784724, 'Cash': 0}
{'Equities': 50.46674802275524, 'Bonds': 49.53325197724476, 'Cash': 0.0}
2015-04-27 00:00:00
{'Equities': 257.370223089183, 'Bonds': 252.60958177784724, 'Cash': 0}
{'Equities': 50.46674802275524, 'Bonds': 49.53325197724476, 'Cash': 0.0}
2015-04-28 00:00:00
{'Equities': 258.1694681275015, 'Bonds': 251.82150292900826, 'Cash': 0}
{'Equities': 50.6223605474174, 'Bonds': 49.3776394525826, 'Cash': 0.0}
2015-04-29 00:00:00
{'Equities': 257.1171288270488, 'Bonds': 251.03342408016925, 'Cash': 0}
{'Equities': 50.59861243012269, 'Bonds': 49.40138756987731, 'Cash': 0.0}
2015-04-30 00:00:00
{'Equities': 254.55954470442973, 'Bonds': 251.3062206047674, 'Cash': 0}
{'Equities': 50.32156001876049, 'Bonds': 49.67843998123951, 'Cash': 0.0}
2015-05-01 00:00:00
{'Equities': 25

{'Equities': 258.10689531485434, 'Bonds': 248.295019077644, 'Cash': 0}
{'Equities': 50.96878348583862, 'Bonds': 49.03121651416137, 'Cash': 0.0}
2015-08-17 00:00:00
{'Equities': 258.10689531485434, 'Bonds': 248.295019077644, 'Cash': 0}
{'Equities': 50.96878348583862, 'Bonds': 49.03121651416137, 'Cash': 0.0}
2015-08-18 00:00:00
{'Equities': 257.31823164384207, 'Bonds': 247.83915306954552, 'Cash': 0}
{'Equities': 50.93823022894881, 'Bonds': 49.061769771051175, 'Cash': 0.0}
2015-08-19 00:00:00
{'Equities': 255.28642015242053, 'Bonds': 248.72049401853593, 'Cash': 0}
{'Equities': 50.6513726249852, 'Bonds': 49.34862737501479, 'Cash': 0.0}
2015-08-20 00:00:00
{'Equities': 249.92618062570986, 'Bonds': 248.93323148898187, 'Cash': 0}
{'Equities': 50.0995219407126, 'Bonds': 49.9004780592874, 'Cash': 0.0}
2015-08-21 00:00:00
{'Equities': 241.99944237299306, 'Bonds': 249.32831536266727, 'Cash': 0}
{'Equities': 49.25417678176273, 'Bonds': 50.74582321823726, 'Cash': 0.0}
2015-08-22 00:00:00
{'Equities

{'Equities': 50.028842879528156, 'Bonds': 49.97115712047184, 'Cash': 0.0}
2015-12-12 00:00:00
{'Equities': 249.73456482289413, 'Bonds': 246.800721839932, 'Cash': 0}
{'Equities': 50.29543146899793, 'Bonds': 49.704568531002074, 'Cash': 0.0}
2015-12-13 00:00:00
{'Equities': 249.73456482289413, 'Bonds': 246.800721839932, 'Cash': 0}
{'Equities': 50.29543146899793, 'Bonds': 49.704568531002074, 'Cash': 0.0}
2015-12-14 00:00:00
{'Equities': 249.73456482289413, 'Bonds': 246.800721839932, 'Cash': 0}
{'Equities': 50.29543146899793, 'Bonds': 49.704568531002074, 'Cash': 0.0}
2015-12-15 00:00:00
{'Equities': 252.3380634297855, 'Bonds': 246.526464956791, 'Cash': 0}
{'Equities': 50.582482632288006, 'Bonds': 49.417517367712, 'Cash': 0.0}
2015-12-16 00:00:00
{'Equities': 255.88096875050363, 'Bonds': 246.2522080736499, 'Cash': 0}
{'Equities': 50.9587855494585, 'Bonds': 49.0412144505415, 'Cash': 0.0}
2015-12-17 00:00:00
{'Equities': 252.15018208702014, 'Bonds': 246.77024885291635, 'Cash': 0}
{'Equities': 

{'Equities': 252.290739101503, 'Bonds': 251.49123282929003, 'Cash': 0}
{'Equities': 50.079350425060746, 'Bonds': 49.920649574939254, 'Cash': 0.0}
2016-03-29 00:00:00
{'Equities': 254.71309828721178, 'Bonds': 252.52969553468182, 'Cash': 0}
{'Equities': 50.21522264871214, 'Bonds': 49.78477735128785, 'Cash': 0.0}
2016-03-30 00:00:00
{'Equities': 255.7821841848151, 'Bonds': 252.43806647244136, 'Cash': 0}
{'Equities': 50.329002800267105, 'Bonds': 49.670997199732895, 'Cash': 0.0}
2016-03-31 00:00:00
{'Equities': 255.17321120516766, 'Bonds': 252.9267548043904, 'Cash': 0}
{'Equities': 50.22106441163735, 'Bonds': 49.77893558836264, 'Cash': 0.0}
2016-04-01 00:00:00
{'Equities': 256.8106718837753, 'Bonds': 252.8554106384159, 'Cash': 0}
{'Equities': 50.38802476572366, 'Bonds': 49.61197523427634, 'Cash': 0.0}
2016-04-02 00:00:00
{'Equities': 255.99870791091197, 'Bonds': 252.8554106384159, 'Cash': 0}
{'Equities': 50.308860354855454, 'Bonds': 49.69113964514455, 'Cash': 0.0}
2016-04-03 00:00:00
{'Equi

{'Equities': 272.3576044495872, 'Bonds': 258.3838815853224, 'Cash': 0}
{'Equities': 51.31643401090241, 'Bonds': 48.683565989097595, 'Cash': 0.0}
2016-08-21 00:00:00
{'Equities': 272.3576044495872, 'Bonds': 258.3838815853224, 'Cash': 0}
{'Equities': 51.31643401090241, 'Bonds': 48.683565989097595, 'Cash': 0.0}
2016-08-22 00:00:00
{'Equities': 272.3576044495872, 'Bonds': 258.3838815853224, 'Cash': 0}
{'Equities': 51.31643401090241, 'Bonds': 48.683565989097595, 'Cash': 0.0}
2016-08-23 00:00:00
{'Equities': 272.8601593750128, 'Bonds': 258.50645458417694, 'Cash': 0}
{'Equities': 51.350640444258154, 'Bonds': 48.64935955574184, 'Cash': 0.0}
2016-08-24 00:00:00
{'Equities': 271.54265051646456, 'Bonds': 258.4758113344633, 'Cash': 0}
{'Equities': 51.23267773884416, 'Bonds': 48.76732226115583, 'Cash': 0.0}
2016-08-25 00:00:00
{'Equities': 271.32532946763183, 'Bonds': 258.2000220870407, 'Cash': 0}
{'Equities': 51.2393464583004, 'Bonds': 48.7606535416996, 'Cash': 0.0}
2016-08-26 00:00:00
{'Equities'

{'Equities': 285.0820139192603, 'Bonds': 247.6168816561927, 'Cash': 0}
{'Equities': 53.51653932214329, 'Bonds': 46.48346067785672, 'Cash': 0.0}
2016-12-14 00:00:00
{'Equities': 282.7243874357749, 'Bonds': 246.35729205967857, 'Cash': 0}
{'Equities': 53.436812952092474, 'Bonds': 46.56318704790752, 'Cash': 0.0}
2016-12-15 00:00:00
{'Equities': 283.7737356394071, 'Bonds': 245.9886316899671, 'Cash': 0}
{'Equities': 53.56623141616508, 'Bonds': 46.43376858383492, 'Cash': 0.0}
2016-12-16 00:00:00
{'Equities': 283.2831312844622, 'Bonds': 246.20368357229881, 'Cash': 0}
{'Equities': 53.50145146883348, 'Bonds': 46.49854853116652, 'Cash': 0.0}
2016-12-17 00:00:00
{'Equities': 283.92364252564033, 'Bonds': 246.84883921929384, 'Cash': 0}
{'Equities': 53.49253254280081, 'Bonds': 46.50746745719921, 'Cash': 0.0}
2016-12-18 00:00:00
{'Equities': 283.92364252564033, 'Bonds': 246.84883921929384, 'Cash': 0}
{'Equities': 53.49253254280081, 'Bonds': 46.50746745719921, 'Cash': 0.0}
2016-12-19 00:00:00
{'Equitie

{'Equities': 295.8108074308845, 'Bonds': 251.49448727289095, 'Cash': 0}
{'Equities': 54.04859231098609, 'Bonds': 45.95140768901392, 'Cash': 0.0}
2017-04-15 00:00:00
{'Equities': 295.8108074308845, 'Bonds': 251.49448727289095, 'Cash': 0}
{'Equities': 54.04859231098609, 'Bonds': 45.95140768901392, 'Cash': 0.0}
2017-04-16 00:00:00
{'Equities': 295.8108074308845, 'Bonds': 251.49448727289095, 'Cash': 0}
{'Equities': 54.04859231098609, 'Bonds': 45.95140768901392, 'Cash': 0.0}
2017-04-17 00:00:00
{'Equities': 295.8108074308845, 'Bonds': 251.49448727289095, 'Cash': 0}
{'Equities': 54.04859231098609, 'Bonds': 45.95140768901392, 'Cash': 0.0}
2017-04-18 00:00:00
{'Equities': 295.000329227187, 'Bonds': 252.38795047648205, 'Cash': 0}
{'Equities': 53.89233569028674, 'Bonds': 46.10766430971327, 'Cash': 0.0}
2017-04-19 00:00:00
{'Equities': 294.3546940479703, 'Bonds': 251.92581433669355, 'Cash': 0}
{'Equities': 53.88343342477455, 'Bonds': 46.11656657522544, 'Cash': 0.0}
2017-04-20 00:00:00
{'Equities'

AttributeError: 'numpy.float64' object has no attribute 'get_value'

In [100]:
portfolio.shares

{<Portfolio.Share at 0x7f68c8a81a58>: 1.3736918706737462,
 <Portfolio.Share at 0x7f68c8a817b8>: 3.0850782324587596}

In [99]:
portfolio.log

{0: ['Buy',
  250.0,
  188.4,
  1.326963906581741,
  '2015-01-01'],
 1: ['Buy',
  250.0,
  82.65,
  3.024803387779794,
  '2015-01-01'],
 2: ['Buy',
  0.17,
  84.08,
  0.002021883920076118,
  Timestamp('2015-02-02 00:00:00', freq='D')],
 3: ['Buy',
  0.162,
  82.65,
  0.0019600725952813067,
  Timestamp('2015-03-02 00:00:00', freq='D')],
 4: ['Buy',
  0.984,
  192.52,
  0.005111157282360274,
  Timestamp('2015-03-23 00:00:00', freq='D')],
 5: ['Buy',
  0.191,
  83.51,
  0.0022871512393725303,
  Timestamp('2015-04-01 00:00:00', freq='D')],
 6: ['Buy',
  0.159,
  82.5,
  0.0019272727272727273,
  Timestamp('2015-05-01 00:00:00', freq='D')],
 7: ['Buy',
  0.166,
  81.9,
  0.002026862026862027,
  Timestamp('2015-06-01 00:00:00', freq='D')],
 8: ['Buy',
  0.902,
  194.27,
  0.0046430225974159675,
  Timestamp('2015-06-22 00:00:00', freq='D')],
 9: ['Buy',
  0.163,
  80.9,
  0.002014833127317676,
  Timestamp('2015-07-01 00:00:00', freq='D')],
 10: ['Buy',
  0.169,
  81.83,
  0.0020652572406207994

In [65]:
portfolio.asset_split['Bonds']-strat.bond_distribution

-2.5185481057840775

In [33]:
portfolio.asset_split['Bonds']

50.29713992195221

In [36]:
246.38667/.502971399

489.8621879690619

In [41]:
(246.38667+243.47552)*.002971399

1.45557602150381

In [37]:
489.86218/2

244.93109

In [39]:
246.38667*.002971399

0.7321131048513301

241.37028324787127

In [54]:
output

In [19]:
portfolio.log

{0: ['Buy',
  250.0,
  247.09,
  1.0117770852725727,
  '2018-01-01'],
 1: ['Buy',
  250.0,
  81.34,
  3.0735185640521268,
  '2018-01-01'],
 2: ['Buy',
  0.177,
  80.16,
  0.002208083832335329,
  Timestamp('2018-02-01 00:00:00', freq='D')],
 3: ['Buy',
  0.165,
  79.52,
  0.002074949698189135,
  Timestamp('2018-03-01 00:00:00', freq='D')],
 4: ['Buy',
  1.084,
  243.53,
  0.004451196977785078,
  Timestamp('2018-03-26 00:00:00', freq='D')]}

In [ ]:
time_period

In [ ]:
start_date

In [ ]:
# Get the value a few days later

portfolio.get_value('2018-05-01')

In [ ]:
portfolio.get_asset_values('2018-05-01')

In [ ]:
portfolio.asset_values

In [ ]:
for share in portfolio.shares:
    print(share.div.index)

In [ ]:
portfolio.reinvest_divs('2018-05-01')

In [ ]:
# import sys
# !{sys.executable} -m pip install pandas_datareader

In [ ]:
## This is useful:
#https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

In [ ]:
bnd = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp = web.DataReader('VOO', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp.head(3)

In [ ]:
bnd.head(3)

In [ ]:
bnd.loc['2017-07-07']['High']

In [ ]:
# Import dividend records
bnd_div = web.DataReader('BND', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')
sp_div = web.DataReader('VOO', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')

In [ ]:
bnd_div.head(3)

In [ ]:
sp_div.head(3)

In [ ]:
sp_div.value.loc['2020-09-29']

In [ ]:
sp_div[sp_div.index < pd.to_datetime('2016-01-01')]

In [ ]:
sp_div.index

In [ ]:
str('sp')+str('_div')

In [ ]:
Assets = {}
Assets['Bonds'] = bnd
Assets['Equities'] = sp

In [ ]:
Dividends

In [ ]:
# Maybe it is worth merging the dividend and share price files together for easier reference?
# Or build a dictionary to allow easy mapping?

## 1. Calculate Value Given a Start and End Date

In [ ]:
start_date = '2015-11-20'
end_date = 

In [ ]:
def calc_value(share, divs, start, end, initial_value):
    # Take the average of the day's high and low for starting share price
    num_shares = initial_value/np.mean([share.loc[start]['Low'],share.loc[start]['High']])
    divs = divs[(divs.index <= pd.to_datetime(end)) & (divs.index >= pd.to_datetime(start))]
    for date in divs.index:
        price = np.mean([share.High.loc[date], share.Low.loc[date]])
        div_value = divs.value.loc[date]
        num_shares += div_value/price
    output = {}
    output['Value'] = num_shares*np.mean([share.loc[end]['Low'],share.loc[end]['High']])
    output['ROI'] = output['Value']-initial_value
    return(output)
    
    # Will need to factor in other actions later, but this will work for now
    
    


In [ ]:
calc_value(sp,sp_div, '2015-11-20', '2017-11-20',100)

In [ ]:
calc_value(bnd,bnd_div, '2015-11-20', '2017-11-20',100)

In [ ]:
sp.Close.plot()

In [ ]:
bnd.Close.plot()

In [ ]:
# Identify the ROI with different asset distributions

In [ ]:
class Share:

    def __init__(self, name, asset_type):
        """Initialize attributes."""
        self.name = name
        self.type = asset_type
        #self.amount = amount
        
    def get_value(self, date):
        df = web.DataReader(self.name, 'yahoo', start=date, end=date)
        return round(df.loc[date,'Close'],2)
    

In [ ]:
bnd = Share('BND','Bond')

In [ ]:
sp = Share('VOO','Equity')

In [ ]:
class Strategy:
    
    def __init__(self,equity_distribution, bond_distribution, cash_distribution, threshold):
        self.equity_distribution = equity_distribution
        self.bond_distribution = bond_distribution
        self.cash_distribution = 100-(equity_distribution+bond_distribution)
        self.threshold = threshold
    

In [ ]:
class Portfolio:
    
    def __init__(self):
        self.shares = {}
        self.log = {}
        self.cash_bal = 0
        self.asset_split = {'Equities': None,'Bonds': None,'Cash': None}
        self.asset_values = {'Equities': None,'Bonds': None,'Cash': None}
        
    def buy(self, share, amount, date):
        purchase_price = share.get_value(date)
        shares = amount/purchase_price
        # Record the transaction
        self.log[len(self.log)] = ['Buy',share,amount,purchase_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] += shares
        elif share not in self.shares:
            self.shares[share] = shares
            
    def sell(self, share, amount, date):
        sell_price = share.get_value(date)
        shares = amount/sell_price
        # Record the transaction
        self.log[len(self.log)] = ['Sell',share,amount,sell_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] -= shares
        elif share not in self.shares:
            self.shares[share] = shares
        cash_bal += amount
        
            
    def get_value(self,date):
        value = self.cash_bal
        # Iterate through the 'shares' attribute and get the current value for each 'Share' object
        for share in self.shares:
            value += share.get_value(date)*self.shares[share]
        return(value)
    
    def get_asset_values(self,date):
        bond_val = 0
        eq_val = 0
        for share in self.shares:
            if share.type == 'Bond':
                bond_val += share.get_value(date)*self.shares[share]
            elif share.type == 'Equity':
                eq_val += share.get_value(date)*self.shares[share]
        total = bond_val+eq_val+self.cash_bal
        self.asset_values = {'Equities': eq_val,'Bonds': bond_val,'Cash': self.cash_bal}
        self.asset_split = {'Equities': (eq_val/total)*100,'Bonds': (bond_val/total)*100,'Cash': (self.cash_bal/total)*100}
            
        
    

In [ ]:
portfolio = Portfolio()

In [ ]:
portfolio.get_value)

In [ ]:
# Calculate portfolio value

start_date = '2015-11-20'
end_date = '2020-11-20'

portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

portfolio.get_value(end_date)

In [ ]:
bond = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
dates = list(bond.index)

In [ ]:
dates = [x.strftime('%Y-%m-%d') for x in dates]

In [ ]:
# Deploy balancing strategy

strategy = Strategy(75,25,0,1)

# Initial Purchase (manually allocated proportions)
portfolio = Portfolio() 
portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

for date in dates:
    portfolio.get_asset_values(date)
    if portfolio.asset_split['Equities'] > strategy.equity_distribution+strategy.threshold:
        amount_to_sell = (portfolio.asset_split['Equities']-strategy.equity_distribution)*portfolio.asset_values['Equities']
        portfolio.sell(sp,amount_to_sell,date)
        portfolio.buy(bnd,amount_to_sell,date)
        

    

In [ ]:
portfolio.get_asset_values('2019-10-29')

In [ ]:
portfolio.asset_split['Equities']